In [6]:
import sys
sys.path.append("..")

In [7]:
from constants import *
from supporting_functions import *
from des_functions import *
from cryptanalysis import *
from collections import Counter

In [8]:
key = format(random.randint(0,2**56-1), '056b')
key = add_parity_key(key)
keys = compute_keys(key)
key

'1001000_1111001_0000111_1101100_1010001_0110010_0000000_0001101_'

In [9]:
# CHARACTERISTIC + 1 Round S-Boxes who hav 0 as input
exp = expansion(THREE_ROUND_CHAR2_OP_XOR[32:])
zero_ix = []
s_box_op = ""
for i in range(8):
    if exp[6*i:6*(i+1)]=='0'*6:
        zero_ix.append(i)
        s_box_op+='0'*4
    else:
        s_box_op+='1'*4
l_prev_mask = permutation(s_box_op)
l_prev_mask = compute_xor(l_prev_mask, '1' * len(l_prev_mask))

In [10]:
# PLAIN TEXT
plain_texts = []
plain_pairs = []
for i in range(int(1e4)):
    pair = get_pair(THREE_ROUND_CHAR2_IP_XOR)
    input_pair = [bits_to_str(inv_initial_permutation(str_to_bits(pair[0]))),
                    bits_to_str(inv_initial_permutation(str_to_bits(pair[1])))]
    plain_pairs.append(input_pair)
    plain_texts.append(input_pair[0])
    plain_texts.append(input_pair[1])
    
# ENCODE
cipher_texts = des_encode_multiple(plain_texts)

# PAIRS
cipher_pairs = []
for i, c in enumerate(cipher_texts):
    c_old = initial_permutation(str_to_bits(c))
    c_new = c_old[32:] + c_old[:32]
    if i%2==0:
        cipher_pairs.append([c_new])
    else:
        cipher_pairs[-1].append(c_new)
        
output_xors = []
for pair in cipher_pairs:
    ans = compute_xor(pair[0], pair[1])
    output_xors.append(ans)

# S-BOX OUTPUT_XOR
s_box_exp_op_xors = []
s_box_ip_pairs = []
for i,op_xor in enumerate(output_xors):
    l_op_xor = op_xor[:32]
    r_op_xor = op_xor[32:]
    r_op_xor = inv_permutation(r_op_xor)
    s_box_xor = r_op_xor
    s_box_exp_op_xors.append(s_box_xor)
    
    r_prev_pair = cipher_pairs[i]
    r_prev_pair = r_prev_pair[0][:32], r_prev_pair[1][:32]
    r_prev_pair = expansion(r_prev_pair[0]), expansion(r_prev_pair[1])
    s_box_ip_pairs.append(r_prev_pair)
    
   
# TESTING FOR EACH BYTE AND EACH KEY
counter = np.zeros([8,64])
for j in range(64):
    candidate_key = format(j, '06b') * 8
    for k in range(len(s_box_exp_op_xors)):
        s_box_op_xor = s_box_exp_op_xors[k]
        s_box_ip_pair = s_box_ip_pairs[k]
        s_box_ip_pair = compute_xor(s_box_ip_pair[0], candidate_key), compute_xor(s_box_ip_pair[1], candidate_key)
        s_box_ip_xor = compute_xor(s_box_ip_pair[0], s_box_ip_pair[1])
        s_box_op_pair = s_boxes_encode(s_box_ip_pair[0]), s_boxes_encode(s_box_ip_pair[1])
        
        masked_exp_op_xor = compute_and(l_prev_mask, s_box_op_xor)
        masked_true_op_xor = compute_and(l_prev_mask, compute_xor(s_box_op_pair[0], s_box_op_pair[1]))
        
        # Rejecting those for which output xor is not possible
        for i in range(8):
            ip_xor = s_box_ip_xor[6*i:6*(i+1)]
            op_xor = s_box_op_xor[4*i:4*(i+1)]
            if op_xor not in ip_xor_to_op_xor[i][ip_xor]:
                continue
            
            if masked_exp_op_xor[4*i:4*(i+1)]==masked_true_op_xor[4*i:4*(i+1)]:
                counter[i][j]+=1
    
for i in range(8):
    predicted_ix = np.argmax(counter[i])
    counter[i].sort()
    print(counter[i])
    print(format(predicted_ix, '06b'))

[ 601.  613.  614.  624.  629.  633.  633.  633.  633.  634.  634.  635.
  639.  640.  641.  641.  643.  646.  647.  648.  650.  652.  652.  655.
  656.  657.  660.  660.  661.  661.  661.  661.  661.  662.  665.  666.
  668.  670.  670.  672.  674.  677.  679.  679.  680.  683.  684.  688.
  691.  692.  692.  692.  692.  696.  700.  708.  710.  714.  716.  720.
  729.  763.  770. 1630.]
101101
[4032. 4034. 4072. 4074. 4076. 4086. 4090. 4092. 4094. 4095. 4102. 4104.
 4104. 4107. 4108. 4108. 4109. 4111. 4117. 4120. 4123. 4124. 4125. 4126.
 4128. 4130. 4131. 4132. 4133. 4134. 4134. 4136. 4141. 4142. 4144. 4146.
 4150. 4152. 4161. 4164. 4164. 4166. 4167. 4168. 4170. 4172. 4175. 4182.
 4183. 4191. 4192. 4194. 4194. 4194. 4198. 4204. 4205. 4207. 4209. 4216.
 4218. 4270. 4283. 4607.]
111011
[1059. 1078. 1085. 1086. 1090. 1091. 1091. 1092. 1093. 1094. 1095. 1095.
 1098. 1098. 1098. 1102. 1107. 1108. 1110. 1110. 1113. 1113. 1114. 1116.
 1116. 1118. 1118. 1120. 1120. 1123. 1127. 1128. 1129. 113